In [ ]:
import pandas as pd

# Description

In [ ]:
#----Setting Up The Info----

#Start with importing: AMT, Sales Period, OnHand & OnOrder File
#Filter AMT to show only "Y", meaning only active skus
#Merge in the OH and OO columns to the AMT on the store and sku columns, save as new dataframe and fillna as 0 
#From the AMT column "old sku" use this to merge in the sales period (usually sales period would be previous year info)
#Fill any null values with 0s
#Create an "Average Weekly Sales" column by with the sales period info.

#----Building The Orders-------

#Info needed to build orders: Truck Size, WOS, Total Ordered Pallets Per Store, and Rank System
#To begin, build a new column called 'AD Units'. AD Units is the sum of on-hands, on-orders, and SOQ units.
#Divide the AD Units by the AWS to build the WOS column. 
#Both the SOQ Units and WOS column will continuely be updating.
#Add a rank column that looks at each store, each item, and ranks based on lowest to highest WOS.
#The lowest WOS is ranked 1 and using a for loop, a pallet will be added to the item with rank 1 until truck size met.
#Pallet will be added to the column called 'Add Pallet'. And another column called 'Total' is part of the for loop check
#The total column will be looking at each store and taking the sum of the column called 'SOQ Pallets'.

# Data Import

In [ ]:
AMT = pd.read_excel(#Insert Current Assortment List Here)
Sales = pd.read_excel(#Insert Sales Time Period (4 Week Period))
Inventory = pd.read_excel(#Insert Current Inventory and Orders Here)

In [ ]:
AMT.head()

In [ ]:
Sales.head()

In [ ]:
Inventory.head()

# Structure Data - Merging, Adding, and Removing Fields

In [ ]:
#Since this AMT File only has active skus, we can skip the filtering step and merge the inventory info and sales info.

SOQ = AMT.merge(Inventory, how = 'left', left_on = ['STR NBR', 'Sku'], right_on = ['Store Nbr', 'SKU Nbr']).fillna(0)

SOQ.drop(columns = ['Store Nbr', 'SKU Name', 'SKU Nbr'], inplace = True)

SOQ = SOQ.merge(Sales, how = 'left', left_on = ['STR NBR', 'Old Sku'], right_on = ['Store Nbr', 'SKU Nbr']).fillna(0)

SOQ.drop(columns = ['Store Nbr', 'SKU Nbr'], inplace = True)

SOQ[['Str OH Units Dly', 'Str OO Units Dly', 'Sales Units']] = SOQ[['Str OH Units Dly', 'Str OO Units Dly', 'Sales Units']].astype('int')

SOQ['AWS'] = SOQ['Sales Units']/4

SOQ['SOQ PLT'] = 0
SOQ['SOQ Units'] = SOQ['SOQ PLT']*SOQ['PLT SZ']
SOQ['AD Units'] = SOQ['Str OH Units Dly'] + SOQ['Str OO Units Dly'] + SOQ['SOQ Units']
SOQ['WOS'] = SOQ['AD Units'] / SOQ['AWS']

SOQ2 = SOQ[['STR NBR', 'Sku', 'WOS']]
SOQ2 = pd.DataFrame.groupby(SOQ2, by = ['STR NBR'],sort = True).rank(method = 'min')

SOQ = SOQ.join(SOQ2, how = 'left', rsuffix='x').fillna(0)

SOQ.drop(columns = 'Skux', inplace = True)

SOQ['WOSx'] = SOQ['WOSx'].astype('int')

SOQ.columns = ['BYO', 'MKT', 'STR', 'Sku', 'Old Sku', 'Sku Description', 'AMT', 'PLT SZ', 'WH', 'Truck Limit', 'STR OH', 'STR OO', 'Sales Units', 'AWS', 'SOQ PLT', 'SOQ Units', 'AD Units', 'WOS', 'Rank']

In [ ]:
SOQ.head()

# Generating Full Truck Quantities

In [ ]:
#----Running While Loop Unitl 50 Pallet Truck Quantities Are Reached----
SOQ_50 = SOQ.loc[SOQ['Truck Limit'] == 18].copy()

x = 0

while x < 50:
    
    SOQ_50['SOQ PLT'] = SOQ_50[['SOQ PLT', 'Rank']].apply((lambda x: x['SOQ PLT']+1 if x['Rank'] == 1 else x['SOQ PLT']),axis =1)

    SOQ_50['SOQ Units'] = SOQ_50['SOQ PLT']*SOQ_50['PLT SZ']
    SOQ_50['AD Units'] = SOQ_50['STR OH'] + SOQ_50['STR OO'] + SOQ_50['SOQ Units']
    SOQ_50['WOS'] = SOQ_50['AD Units'] / SOQ_50['AWS']

    SOQ2 = SOQ_50[['STR', 'Sku', 'WOS']]
    SOQ2 = pd.DataFrame.groupby(SOQ2, by = ['STR'],sort = True).rank(method = 'min')

    SOQ_50 = SOQ_50.join(SOQ2, how = 'left', rsuffix='x').fillna(0)

    SOQ_50.drop(columns = 'Skux', inplace = True)
    SOQ_50['Rank'] = SOQ_50['WOSx'].astype('int')
    SOQ_50.drop(columns = 'WOSx', inplace = True)
    
    x += 1
    
#----Running While Loop Unitl 51 Pallet Truck Quantities Are Reached----    
SOQ_51 = SOQ.loc[SOQ['Truck Limit'] == 20].copy()

x = 0

while x < 51:
    
    SOQ_51['SOQ PLT'] = SOQ_51[['SOQ PLT', 'Rank']].apply((lambda x: x['SOQ PLT']+1 if x['Rank'] == 1 else x['SOQ PLT']),axis =1)

    SOQ_51['SOQ Units'] = SOQ_51['SOQ PLT']*SOQ_51['PLT SZ']
    SOQ_51['AD Units'] = SOQ_51['STR OH'] + SOQ_51['STR OO'] + SOQ_51['SOQ Units']
    SOQ_51['WOS'] = SOQ_51['AD Units'] / SOQ_51['AWS']

    SOQ2 = SOQ_51[['STR', 'Sku', 'WOS']]
    SOQ2 = pd.DataFrame.groupby(SOQ2, by = ['STR'],sort = True).rank(method = 'min')

    SOQ_51 = SOQ_51.join(SOQ2, how = 'left', rsuffix='x').fillna(0)

    SOQ_51.drop(columns = 'Skux', inplace = True)
    SOQ_51['Rank'] = SOQ_51['WOSx'].astype('int')
    SOQ_51.drop(columns = 'WOSx', inplace = True)
    
    x += 1
    
#----Running While Loop Unitl 52 Pallet Truck Quantities Are Reached----
SOQ_52 = SOQ.loc[SOQ['Truck Limit'] == 22].copy()

x = 0

while x < 52:
    
    SOQ_52['SOQ PLT'] = SOQ_52[['SOQ PLT', 'Rank']].apply((lambda x: x['SOQ PLT']+1 if x['Rank'] == 1 else x['SOQ PLT']),axis =1)

    SOQ_52['SOQ Units'] = SOQ_52['SOQ PLT']*SOQ_52['PLT SZ']
    SOQ_52['AD Units'] = SOQ_52['STR OH'] + SOQ_52['STR OO'] + SOQ_52['SOQ Units']
    SOQ_52['WOS'] = SOQ_52['AD Units'] / SOQ_52['AWS']

    SOQ2 = SOQ_52[['STR', 'Sku', 'WOS']]
    SOQ2 = pd.DataFrame.groupby(SOQ2, by = ['STR'],sort = True).rank(method = 'min')

    SOQ_52 = SOQ_52.join(SOQ2, how = 'left', rsuffix='x').fillna(0)

    SOQ_52.drop(columns = 'Skux', inplace = True)
    SOQ_52['Rank'] = SOQ_52['WOSx'].astype('int')
    SOQ_52.drop(columns = 'WOSx', inplace = True)
    
    x += 1

# Adding In Totals Column

In [ ]:
#----For 50 Pallet Orders----
SOQ3 =  SOQ_50.groupby(by = 'STR')
Total = SOQ3['SOQ PLT'].sum()

Total = pd.Series.to_frame(Total)

SOQ_50 = SOQ_50.merge(Total, how = 'left', on = 'STR', suffixes=('', ' Total'))

#----For 51 Pallet Orders----
SOQ3 =  SOQ_51.groupby(by = 'STR')
Total = SOQ3['SOQ PLT'].sum()

Total = pd.Series.to_frame(Total)

SOQ_51 = SOQ_51.merge(Total, how = 'left', on = 'STR', suffixes=('', ' Total'))

#----For 52 Pallet Orders----
SOQ3 =  SOQ_52.groupby(by = 'STR')
Total = SOQ3['SOQ PLT'].sum()

Total = pd.Series.to_frame(Total)

SOQ_52 = SOQ_52.merge(Total, how = 'left', on = 'STR', suffixes=('', ' Total'))

# Exporting The Sheets On Same File

In [ ]:
#Change File Name Every Time Code Is Run

export_file = pd.ExcelWriter('19 5.21 SOQ Reports.xlsx', engine = 'xlsxwriter')

SOQ_50.to_excel(export_file, sheet_name= 'Sheet 1', index = False)
SOQ_51.to_excel(export_file, sheet_name= 'Sheet 2',index = False)
SOQ_52.to_excel(export_file, sheet_name= 'Sheet 3', index = False)

export_file.save()